# 1. Loading and Reading Data 🛄👀

## 1.1 API Access fitbit 📲

The activity data recorded by fitbit bands, can be accessed through an API developed for this purpose: https://dev.fitbit.com/build/reference/web-api/

**Important NOTE**: The 'token' parameter is provided to me by the fitbit API itself (**only active for a week**), so it may not work when reviewing this exercise

In [30]:
import requests
import json
import pandas as pd
from time import sleep
from datetime import datetime

#Automated process with a function to access each of variables more quickly.
#I introduce sleep() to try to save API data limit restrictions
def df_fitbit(activity, base_date, end_date, token):
    url = 'https://api.fitbit.com/1/user/-/' + activity + '/date/' + base_date + '/' + end_date + '.json'
    response = requests.get(url=url, headers={'Authorization':'Bearer ' + token}).json()
    sleep(30)
    return response

token = 'personal token number'
base_date = '2018-07-31'
end_date = '2016-02-16'
activity = ['activities/log/steps', 'activities/log/distance', 'activities/log/calories', 'activities/log/floors', 
            'activities/log/elevation', 'activities/minutesSedentary', 'activities/log/minutesLightlyActive', 
            'activities/log/minutesFairlyActive', 'activities/log/minutesVeryActive', 'activities/log/activityCalories', 
            'sleep/startTime', 'sleep/timeInBed', 'sleep/minutesAsleep', 'sleep/awakeningsCount', 'sleep/minutesAwake', 
            'sleep/minutesToFallAsleep', 'sleep/minutesAfterWakeup', 'sleep/efficiency']

df = pd.DataFrame()

data_fitbit_steps = df_fitbit(activity[0], base_date, end_date, token)
df['date'] = pd.DataFrame(data_fitbit_steps['activities-log-steps'])['dateTime']
df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
df['steps'] = pd.DataFrame(data_fitbit_steps['activities-log-steps'])['value'].astype(int)

data_fitbit_dist = df_fitbit(activity[1], base_date, end_date, token)
df['dist'] = pd.DataFrame(data_fitbit_dist['activities-log-distance'])['value'].astype(float)

data_fitbit_cal = df_fitbit(activity[2], base_date, end_date, token)
df['calories'] = pd.DataFrame(data_fitbit_cal['activities-log-calories'])['value'].astype(int)

data_fitbit_floors = df_fitbit(activity[3], base_date, end_date, token)
df['floors'] = pd.DataFrame(data_fitbit_floors['activities-log-floors'])['value'].astype(int)

data_fitbit_elevation = df_fitbit(activity[4], base_date, end_date, token)
df['elevation'] = pd.DataFrame(data_fitbit_elevation['activities-log-elevation'])['value'].astype(float)

df.head(10)

,date,steps,dist,calories,floors,elevation
0,2016-02-16,6958,5.21670,2390,25,76.0
1,2016-02-17,13881,10.33667,3063,28,85.0
2,2016-02-18,8704,6.46707,2514,21,64.0
3,2016-02-19,14988,11.15941,3244,26,79.0
4,2016-02-20,420,0.31206,2204,0,0.0
5,2016-02-21,7707,5.72645,2478,0,0.0
6,2016-02-22,10315,7.66404,2629,26,79.0
7,2016-02-23,8608,6.39598,2499,18,54.0
8,2016-02-24,44,0.03270,1987,0,0.0
9,2016-02-25,12312,9.14798,2823,19,57.0


## IMPORTANT NOTE: 

With the API usage strategy, I've managed to get recent data from months or 1 year back, without problem. But when I try to get data beyond a year (30 months) i had problem becouse the data download  have limit by the fitbit API.  
My goal was to take out all the data from the beginning of the use of the band that goes back to February 2016 so I had to change strategy to get all the data. Within my fitbit user allows me to download in csv the data from month to month (activity and sleep), so I try to download them, one by one and treat them as csv (In total I download 60 csv files, 30 with activity variables and 30 with sleep variables). This strategy has meant more work but I get a broader data history.

## 1.2 Csv Access 💾

By fitbit website with my user's profile you can export in csv the data recorded by the band with the limitation of 1 month for each file, so 60 csv files (30 of physical activity and 30 of sleep log) have been downloaded. I have data recorded until **February 2016**. I'll collect until that month.  

To organize the data load, the payload has been divided into two dataframes to address its different variables:
- 1.2.1 Upload 30 csv of physical activity.
- 1.2.2 Loading 30 csv sleep records.

### 1.2.1 Manage **physical activity** csv🏃‍

In [1]:
import requests
import json
import pandas as pd
import glob
import numpy as np
from datetime import datetime
import glob

**I use a function to join/concatenate cvs organized into folders**.  
Downloaded to two folders (activity and sleep)

In [2]:
def unir_csvs(path):
    #import glob
    allFiles = glob.glob(path + "/*.csv")
    df = pd.DataFrame()
    list_csv = []
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=1, thousands='.', decimal=',')
        list_csv.append(df)
        df = pd.concat(list_csv)
    return df

In [3]:
path_activity =r'path csv files'
df_activity = unir_csvs(path_activity)
df_activity.head()

,Fecha,Calorías quemadas,Pasos,Distancia,Plantas,Minutos de actividad nula,Minutos de actividad ligera,Minutos de actividad media,Minutos de actividad alta,Calorías por actividad
0,01-07-2018,2825,12457,9.26,9,552,274,53,30,1540
1,02-07-2018,2305,5251,3.90,3,930,90,14,30,765
2,03-07-2018,2658,8430,6.30,18,1225,131,30,54,1200
3,04-07-2018,2459,6781,5.04,18,845,166,12,22,966
4,05-07-2018,2725,6643,4.80,13,814,127,42,46,1246


In [4]:
#Review dataset
df_activity.describe()
df_activity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 912 entries, 0 to 28
Data columns (total 10 columns):
Fecha                          912 non-null object
Calorías quemadas              912 non-null int64
Pasos                          912 non-null int64
Distancia                      912 non-null float64
Plantas                        912 non-null int64
Minutos de actividad nula      912 non-null int64
Minutos de actividad ligera    912 non-null int64
Minutos de actividad media     912 non-null int64
Minutos de actividad alta      912 non-null int64
Calorías por actividad         912 non-null int64
dtypes: float64(1), int64(8), object(1)
memory usage: 78.4+ KB


I'm trying to clean up some loaded variables from the csv´s

In [5]:
#I try to detect variables that are not numeric although with df.info() I already have clues. 
cols = df_activity.columns[df_activity.dtypes.eq(object)] #I note which variables are of type 'object'
cols #the variable 'Date' converse to dtype 'Date'.

Index(['Fecha'], dtype='object')

In [7]:
#Change 'Fecha' variable to 'datetime'
df_activity['Fecha'] = pd.to_datetime(df_activity['Fecha'], format="%d-%m-%Y")
df_activity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 912 entries, 0 to 28
Data columns (total 10 columns):
Fecha                          912 non-null datetime64[ns]
Calorías quemadas              912 non-null int64
Pasos                          912 non-null int64
Distancia                      912 non-null float64
Plantas                        912 non-null int64
Minutos de actividad nula      912 non-null int64
Minutos de actividad ligera    912 non-null int64
Minutos de actividad media     912 non-null int64
Minutos de actividad alta      912 non-null int64
Calorías por actividad         912 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(8)
memory usage: 78.4 KB


In [8]:
#Rename the columns
df_activity = df_activity.rename({'Fecha':'date', 'Calorías quemadas':'calories', 'Pasos':'steps', 'Distancia':'dist', 'Plantas':'floors', 'Minutos de actividad nula':'sedant', 'Minutos de actividad ligera':'active_light', 'Minutos de actividad media':'active_fair', 'Minutos de actividad alta':'active_very', 'Calorías por actividad':'active_cals'}, axis='columns')
df_activity.head()

,date,calories,steps,dist,floors,sedant,active_light,active_fair,active_very,active_cals
0,2018-07-01,2825,12457,9.26,9,552,274,53,30,1540
1,2018-07-02,2305,5251,3.90,3,930,90,14,30,765
2,2018-07-03,2658,8430,6.30,18,1225,131,30,54,1200
3,2018-07-04,2459,6781,5.04,18,845,166,12,22,966
4,2018-07-05,2725,6643,4.80,13,814,127,42,46,1246


In [10]:
#Reset the index
df_activity = df_activity.reset_index(drop=True)
df_activity.head()

,date,calories,steps,dist,floors,sedant,active_light,active_fair,active_very,active_cals
0,2018-07-01,2825,12457,9.26,9,552,274,53,30,1540
1,2018-07-02,2305,5251,3.90,3,930,90,14,30,765
2,2018-07-03,2658,8430,6.30,18,1225,131,30,54,1200
3,2018-07-04,2459,6781,5.04,18,845,166,12,22,966
4,2018-07-05,2725,6643,4.80,13,814,127,42,46,1246


### 1.2.2 Manage sleep csv records 🛏🌛

In [12]:
import requests
import json
import pandas as pd
import glob
import numpy as np
from datetime import datetime
import glob

In [13]:
#I use the function developed above for reading and joining several csv: 'unir_csvs(path)'
path1 =r'path csv files'
df_sleep = unir_csvs(path1)
df_sleep.head()
df_sleep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 727 entries, 0 to 14
Data columns (total 9 columns):
Hora de inicio                           727 non-null object
Hora de finalización                     727 non-null object
Minutos dormido                          727 non-null int64
Minutos despierto                        727 non-null int64
Número de veces que te has despertado    727 non-null int64
Tiempo que he estado en la cama          727 non-null int64
Minutos de fase REM                      727 non-null object
Minutos de sueño ligero                  727 non-null object
Minutos de sueño profundo                727 non-null object
dtypes: int64(4), object(5)
memory usage: 56.8+ KB


In [15]:
#I try again to define which variables are of type 'object' to pass to 'numeric' or 'datetime'
cols = df_sleep.columns[df_sleep.dtypes.eq(object)]
cols

Index(['Hora de inicio', 'Hora de finalización', 'Minutos de fase REM',
       'Minutos de sueño ligero', 'Minutos de sueño profundo'],
      dtype='object')

In [16]:
#There are 3 other Variables/Series with type 'object' to numeric type. 
def coerce_df_columns_to_numeric(df, column_list):
    df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')
    return df

coerce_df_columns_to_numeric(df_sleep, ['Minutos de fase REM', 'Minutos de sueño ligero', 'Minutos de sueño profundo'])
df_sleep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 727 entries, 0 to 14
Data columns (total 9 columns):
Hora de inicio                           727 non-null object
Hora de finalización                     727 non-null object
Minutos dormido                          727 non-null int64
Minutos despierto                        727 non-null int64
Número de veces que te has despertado    727 non-null int64
Tiempo que he estado en la cama          727 non-null int64
Minutos de fase REM                      150 non-null float64
Minutos de sueño ligero                  150 non-null float64
Minutos de sueño profundo                150 non-null float64
dtypes: float64(3), int64(4), object(2)
memory usage: 56.8+ KB


📆 I have to take special care in the **treatment of the Date** because it will be the key to joining the different datasets

In [17]:
#I create a 'Fecha' variable that tells us the reference date of the record. I remove it from the variable 'Hora de inicio'
df_sleep['Fecha'] = df_sleep['Hora de inicio'].map(lambda x: x[0:10])
df_sleep.head()

,Hora de inicio,Hora de finalización,Minutos dormido,Minutos despierto,Número de veces que te has despertado,Tiempo que he estado en la cama,Minutos de fase REM,Minutos de sueño ligero,Minutos de sueño profundo,Fecha
0,17-07-2018 1:04AM,17-07-2018 7:46AM,360,42,29,402,101.0,191.0,68.0,17-07-2018
1,16-07-2018 12:52AM,16-07-2018 7:35AM,346,56,20,402,72.0,181.0,93.0,16-07-2018
2,15-07-2018 2:50AM,15-07-2018 10:28AM,392,65,30,457,57.0,296.0,39.0,15-07-2018
3,13-07-2018 1:16AM,13-07-2018 7:24AM,299,68,15,367,25.0,202.0,72.0,13-07-2018
4,11-07-2018 1:03AM,11-07-2018 7:54AM,358,52,36,410,74.0,207.0,77.0,11-07-2018


In [19]:
#Change the order in the 'Fecha' column to put it first
cols = df_sleep.columns.tolist()
column_to_move = "Fecha"
new_position = 0
cols
cols.insert(new_position, cols.pop(cols.index(column_to_move)))
df_sleep = df_sleep[cols]
df_sleep.head()

,Fecha,Hora de inicio,Hora de finalización,Minutos dormido,Minutos despierto,Número de veces que te has despertado,Tiempo que he estado en la cama,Minutos de fase REM,Minutos de sueño ligero,Minutos de sueño profundo
0,17-07-2018,17-07-2018 1:04AM,17-07-2018 7:46AM,360,42,29,402,101.0,191.0,68.0
1,16-07-2018,16-07-2018 12:52AM,16-07-2018 7:35AM,346,56,20,402,72.0,181.0,93.0
2,15-07-2018,15-07-2018 2:50AM,15-07-2018 10:28AM,392,65,30,457,57.0,296.0,39.0
3,13-07-2018,13-07-2018 1:16AM,13-07-2018 7:24AM,299,68,15,367,25.0,202.0,72.0
4,11-07-2018,11-07-2018 1:03AM,11-07-2018 7:54AM,358,52,36,410,74.0,207.0,77.0


In [20]:
repefechas = df_sleep.groupby(df_sleep['Fecha'], as_index=False).size()
repefechas = repefechas[repefechas > 1].reset_index()
repefechas.info()
#There are 115 dates that are repeated at least twice

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 2 columns):
Fecha    115 non-null object
0        115 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [21]:
#We group and add up to get non-repeated dates so we can cross with the other datasets
df_sleep_grouped = df_sleep.groupby('Fecha', as_index=False).sum(min_count=1) # min_count=1, keeps the "Nan" values
df_sleep_grouped.head()

,Fecha,Minutos dormido,Minutos despierto,Número de veces que te has despertado,Tiempo que he estado en la cama,Minutos de fase REM,Minutos de sueño ligero,Minutos de sueño profundo
0,01-01-2017,473,32,2,505,NaN,NaN,NaN
1,01-01-2018,388,81,43,469,63.0,252.0,73.0
2,01-02-2018,394,40,3,435,NaN,NaN,NaN
3,01-03-2016,387,41,4,433,NaN,NaN,NaN
4,01-03-2017,372,53,4,425,NaN,NaN,NaN


In [22]:
#Change 'Fecha' column to the datetime type
df_sleep_grouped['Fecha'] = pd.to_datetime(df_sleep_grouped['Fecha'], format="%d-%m-%Y")
df_sleep_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 608 entries, 0 to 607
Data columns (total 8 columns):
Fecha                                    608 non-null datetime64[ns]
Minutos dormido                          608 non-null int64
Minutos despierto                        608 non-null int64
Número de veces que te has despertado    608 non-null int64
Tiempo que he estado en la cama          608 non-null int64
Minutos de fase REM                      146 non-null float64
Minutos de sueño ligero                  146 non-null float64
Minutos de sueño profundo                146 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(4)
memory usage: 42.8 KB


In [23]:
#Mixed dates appear in different years
#Order the date from the most recent to the oldest
df_sleep_grouped = df_sleep_grouped.sort_values(by=['Fecha'], ascending=False)
df_sleep_grouped

,Fecha,Minutos dormido,Minutos despierto,Número de veces que te has despertado,Tiempo que he estado en la cama,Minutos de fase REM,Minutos de sueño ligero,Minutos de sueño profundo
344,2018-07-17,360,42,29,402,101.0,191.0,68.0
321,2018-07-16,346,56,20,402,72.0,181.0,93.0
297,2018-07-15,392,65,30,457,57.0,296.0,39.0
261,2018-07-13,299,68,15,367,25.0,202.0,72.0
217,2018-07-11,358,52,36,410,74.0,207.0,77.0
176,2018-07-09,334,60,24,394,68.0,168.0,98.0
155,2018-07-08,80,2,0,82,NaN,NaN,NaN
135,2018-07-07,358,17,0,376,NaN,NaN,NaN
113,2018-07-06,413,56,29,469,65.0,223.0,59.0
93,2018-07-05,365,46,33,411,81.0,212.0,72.0


In [24]:
#Rename variables to make it easier to process columns and normalize it with the activity dataset
df_sleep_grouped = df_sleep_grouped.rename({'Fecha':'date', 'Minutos dormido':'sleep_minutesAsleep', 'Minutos despierto':'sleep_minutesAwake', 'Número de veces que te has despertado':'sleep_awakeningsCount', 'Tiempo que he estado en la cama':'sleep_timeInBed', 'Minutos de fase REM':'sleep_REM', 'Minutos de sueño ligero':'sleep_light', 'Minutos de sueño profundo':'sleep_deep'}, axis='columns')
df_sleep_grouped.head()

,date,sleep_minutesAsleep,sleep_minutesAwake,sleep_awakeningsCount,sleep_timeInBed,sleep_REM,sleep_light,sleep_deep
344,2018-07-17,360,42,29,402,101.0,191.0,68.0
321,2018-07-16,346,56,20,402,72.0,181.0,93.0
297,2018-07-15,392,65,30,457,57.0,296.0,39.0
261,2018-07-13,299,68,15,367,25.0,202.0,72.0
217,2018-07-11,358,52,36,410,74.0,207.0,77.0


In [25]:
#Reset the index to maintain an order
df_sleep_grouped = df_sleep_grouped.reset_index(drop=True)
df_sleep_grouped.head()

,date,sleep_minutesAsleep,sleep_minutesAwake,sleep_awakeningsCount,sleep_timeInBed,sleep_REM,sleep_light,sleep_deep
0,2018-07-17,360,42,29,402,101.0,191.0,68.0
1,2018-07-16,346,56,20,402,72.0,181.0,93.0
2,2018-07-15,392,65,30,457,57.0,296.0,39.0
3,2018-07-13,299,68,15,367,25.0,202.0,72.0
4,2018-07-11,358,52,36,410,74.0,207.0,77.0


🌛🤦‍ There are two sleep variables that are extracted from the **fitbit API** but do not appear in the export to csv's.
I try to extract it from the API (Start Time of day:'startTime', Sleep Efficiency:'sleep_efficiency').

#### A) API access📲: Removing the variable over the start time of sleep: 'sleep_start'

In [26]:
import requests
import json
import pandas as pd
from time import sleep
from datetime import datetime

In [27]:
#API connection requires a token supplied by fitbit after an API registration: https://dev.fitbit.com/apps/new
token = 'personal token'

url_startTime = 'https://api.fitbit.com/1/user/-/sleep/startTime/date/2018-07-31/2016-02-01.json'
response = requests.get(url=url_startTime, headers={'Authorization':'Bearer ' + token}).json()
response

{'sleep-startTime': [{'dateTime': '2016-02-01', 'value': ''},
  {'dateTime': '2016-02-02', 'value': ''},
  {'dateTime': '2016-02-03', 'value': ''},
  {'dateTime': '2016-02-04', 'value': ''},
  {'dateTime': '2016-02-05', 'value': ''},
  {'dateTime': '2016-02-06', 'value': ''},
  {'dateTime': '2016-02-07', 'value': ''},
  {'dateTime': '2016-02-08', 'value': ''},
  {'dateTime': '2016-02-09', 'value': ''},
  {'dateTime': '2016-02-10', 'value': ''},
  {'dateTime': '2016-02-11', 'value': ''},
  {'dateTime': '2016-02-12', 'value': ''},
  {'dateTime': '2016-02-13', 'value': ''},
  {'dateTime': '2016-02-14', 'value': ''},
  {'dateTime': '2016-02-15', 'value': ''},
  {'dateTime': '2016-02-16', 'value': '00:40'},
  {'dateTime': '2016-02-17', 'value': '00:47'},
  {'dateTime': '2016-02-18', 'value': '00:24'},
  {'dateTime': '2016-02-19', 'value': '02:20'},
  {'dateTime': '2016-02-20', 'value': '03:26'},
  {'dateTime': '2016-02-21', 'value': '01:28'},
  {'dateTime': '2016-02-22', 'value': '01:05'},


In [38]:
#Extract the values from the json file
df_startTime = pd.DataFrame()
df_startTime['date'] = pd.DataFrame(response['sleep-startTime'])['dateTime']#Generate date key to join with other df
df_startTime['date'] = pd.to_datetime(df_startTime['date'],infer_datetime_format=True)# Change type 'datetime'
df_startTime['sleep_start'] = pd.DataFrame(response['sleep-startTime'])['value']#Extract values from the variable
print(df_startTime[20:30])
df_startTime.info()

         date sleep_start
20 2016-02-21       01:28
21 2016-02-22       01:05
22 2016-02-23       00:24
23 2016-02-24            
24 2016-02-25       00:36
25 2016-02-26       00:46
26 2016-02-27       00:48
27 2016-02-28       23:58
28 2016-02-29            
29 2016-03-01       00:27
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 2 columns):
date           912 non-null datetime64[ns]
sleep_start    912 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 14.3+ KB


In [40]:
#Normalize the variable 'sleep_start' to a single number to see the start time of the sleep and make operations
df_startTime['sleep_start']=df_startTime['sleep_start'].replace('',np.nan) #Convert null values to 'Nan'

In [42]:
#Change each variable to the 'datetime' type and calculate the normalized time
df_startTime['sleep_start_hr'] = round(df_startTime['sleep_start'].map(lambda x: (datetime.strptime(str(x),"%H:%M")).hour+(datetime.strptime(str(x),"%H:%M")).minute/60.0, na_action = 'ignore'), 2)
df_startTime[20:30]

,date,sleep_start,sleep_start_hr
20,2016-02-21,01:28,1.47
21,2016-02-22,01:05,1.08
22,2016-02-23,00:24,0.40
23,2016-02-24,NaN,NaN
24,2016-02-25,00:36,0.60
25,2016-02-26,00:46,0.77
26,2016-02-27,00:48,0.80
27,2016-02-28,23:58,23.97
28,2016-02-29,NaN,NaN
29,2016-03-01,00:27,0.45


#### B) 📲API Access: Removing the variable sleep efficiency 🛏⏳. 'sleep_efficiency'

In [43]:
#Same procedure as the previous connection, it just changes the url of where the variable is obtained
token = 'personal token'

url_sleepefficiency = 'https://api.fitbit.com/1/user/-/sleep/efficiency/date/2018-07-31/2016-02-01.json'
response2 = requests.get(url=url_sleepefficiency, headers={'Authorization':'Bearer ' + token}).json()
response2

{'sleep-efficiency': [{'dateTime': '2016-02-01', 'value': '0'},
  {'dateTime': '2016-02-02', 'value': '0'},
  {'dateTime': '2016-02-03', 'value': '0'},
  {'dateTime': '2016-02-04', 'value': '0'},
  {'dateTime': '2016-02-05', 'value': '0'},
  {'dateTime': '2016-02-06', 'value': '0'},
  {'dateTime': '2016-02-07', 'value': '0'},
  {'dateTime': '2016-02-08', 'value': '0'},
  {'dateTime': '2016-02-09', 'value': '0'},
  {'dateTime': '2016-02-10', 'value': '0'},
  {'dateTime': '2016-02-11', 'value': '0'},
  {'dateTime': '2016-02-12', 'value': '0'},
  {'dateTime': '2016-02-13', 'value': '0'},
  {'dateTime': '2016-02-14', 'value': '0'},
  {'dateTime': '2016-02-15', 'value': '0'},
  {'dateTime': '2016-02-16', 'value': '100'},
  {'dateTime': '2016-02-17', 'value': '90'},
  {'dateTime': '2016-02-18', 'value': '93'},
  {'dateTime': '2016-02-19', 'value': '87'},
  {'dateTime': '2016-02-20', 'value': '92'},
  {'dateTime': '2016-02-21', 'value': '91'},
  {'dateTime': '2016-02-22', 'value': '91'},
  {'

In [44]:
#Extract the date and values to a new dataframe from the json file
df_efficiencysleep = pd.DataFrame()
df_efficiencysleep['date'] = pd.DataFrame(response2['sleep-efficiency'])['dateTime']
df_efficiencysleep['date'] = pd.to_datetime(df_efficiencysleep['date'],infer_datetime_format=True)
df_efficiencysleep['sleep_efficiency'] = pd.DataFrame(response2['sleep-efficiency'])['value']
df_efficiencysleep[15:20]

,date,sleep_efficiency
15,2016-02-16,100
16,2016-02-17,90
17,2016-02-18,93
18,2016-02-19,87
19,2016-02-20,92


In [45]:
df_efficiencysleep.info() #Check type 'object' variable, pass to numeric

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 2 columns):
date                912 non-null datetime64[ns]
sleep_efficiency    912 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 14.3+ KB


In [46]:
#Pass to numeric type. Use function defined above
def coerce_df_columns_to_numeric(df, column_list):
    df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')
    return df

df_efficiencysleep = coerce_df_columns_to_numeric(df_efficiencysleep, ['sleep_efficiency'])
df_efficiencysleep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 2 columns):
date                912 non-null datetime64[ns]
sleep_efficiency    912 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 14.3 KB


In [47]:
#Note few 0 values appear. Treat them like Nan as they are actually missing values
df_efficiencysleep['sleep_efficiency']=df_efficiencysleep['sleep_efficiency'].replace(0,np.nan)
df_efficiencysleep.head()

,date,sleep_efficiency
0,2016-02-01,NaN
1,2016-02-02,NaN
2,2016-02-03,NaN
3,2016-02-04,NaN
4,2016-02-05,NaN


### 1.3 Join dataframes: Activity, sleep, sleep start and sleep efficiency 👪

In [48]:
df_JLFN_fitbit = pd.merge(df_activity, df_sleep_grouped, how='left', on='date')
df_JLFN_fitbit.head()

,date,calories,steps,dist,floors,sedant,active_light,active_fair,active_very,active_cals,sleep_minutesAsleep,sleep_minutesAwake,sleep_awakeningsCount,sleep_timeInBed,sleep_REM,sleep_light,sleep_deep
0,2018-07-01,2825,12457,9.26,9,552,274,53,30,1540,493.0,38.0,1.0,531.0,NaN,NaN,NaN
1,2018-07-02,2305,5251,3.90,3,930,90,14,30,765,336.0,40.0,3.0,376.0,NaN,NaN,NaN
2,2018-07-03,2658,8430,6.30,18,1225,131,30,54,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-04,2459,6781,5.04,18,845,166,12,22,966,322.0,73.0,22.0,395.0,58.0,171.0,93.0
4,2018-07-05,2725,6643,4.80,13,814,127,42,46,1246,365.0,46.0,33.0,411.0,81.0,212.0,72.0


In [49]:
df_JLFN_fitbit = pd.merge(df_JLFN_fitbit, df_startTime, how='left', on='date')
df_JLFN_fitbit.head()

,date,calories,steps,dist,floors,sedant,active_light,active_fair,active_very,active_cals,sleep_minutesAsleep,sleep_minutesAwake,sleep_awakeningsCount,sleep_timeInBed,sleep_REM,sleep_light,sleep_deep,sleep_start,sleep_start_hr
0,2018-07-01,2825,12457,9.26,9,552,274,53,30,1540,493.0,38.0,1.0,531.0,NaN,NaN,NaN,01:08,1.13
1,2018-07-02,2305,5251,3.90,3,930,90,14,30,765,336.0,40.0,3.0,376.0,NaN,NaN,NaN,01:47,1.78
2,2018-07-03,2658,8430,6.30,18,1225,131,30,54,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-04,2459,6781,5.04,18,845,166,12,22,966,322.0,73.0,22.0,395.0,58.0,171.0,93.0,01:12,1.20
4,2018-07-05,2725,6643,4.80,13,814,127,42,46,1246,365.0,46.0,33.0,411.0,81.0,212.0,72.0,01:06,1.10


In [50]:
#Change order of the 'sleep_start' column to the start of sleep-related variables. Column 11
cols = df_JLFN_fitbit.columns.tolist()
column_to_move = "sleep_start_hr"
new_position = 10
cols
cols.insert(new_position, cols.pop(cols.index(column_to_move)))
df_JLFN_fitbit = df_JLFN_fitbit[cols]
df_JLFN_fitbit.head()

,date,calories,steps,dist,floors,sedant,active_light,active_fair,active_very,active_cals,sleep_start_hr,sleep_minutesAsleep,sleep_minutesAwake,sleep_awakeningsCount,sleep_timeInBed,sleep_REM,sleep_light,sleep_deep,sleep_start
0,2018-07-01,2825,12457,9.26,9,552,274,53,30,1540,1.13,493.0,38.0,1.0,531.0,NaN,NaN,NaN,01:08
1,2018-07-02,2305,5251,3.90,3,930,90,14,30,765,1.78,336.0,40.0,3.0,376.0,NaN,NaN,NaN,01:47
2,2018-07-03,2658,8430,6.30,18,1225,131,30,54,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-04,2459,6781,5.04,18,845,166,12,22,966,1.20,322.0,73.0,22.0,395.0,58.0,171.0,93.0,01:12
4,2018-07-05,2725,6643,4.80,13,814,127,42,46,1246,1.10,365.0,46.0,33.0,411.0,81.0,212.0,72.0,01:06


In [51]:
df_JLFN_fitbit = pd.merge(df_JLFN_fitbit, df_efficiencysleep, how='left', on='date')
df_JLFN_fitbit.head()

,date,calories,steps,dist,floors,sedant,active_light,active_fair,active_very,active_cals,sleep_start_hr,sleep_minutesAsleep,sleep_minutesAwake,sleep_awakeningsCount,sleep_timeInBed,sleep_REM,sleep_light,sleep_deep,sleep_start,sleep_efficiency
0,2018-07-01,2825,12457,9.26,9,552,274,53,30,1540,1.13,493.0,38.0,1.0,531.0,NaN,NaN,NaN,01:08,93.0
1,2018-07-02,2305,5251,3.90,3,930,90,14,30,765,1.78,336.0,40.0,3.0,376.0,NaN,NaN,NaN,01:47,89.0
2,2018-07-03,2658,8430,6.30,18,1225,131,30,54,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-04,2459,6781,5.04,18,845,166,12,22,966,1.20,322.0,73.0,22.0,395.0,58.0,171.0,93.0,01:12,82.0
4,2018-07-05,2725,6643,4.80,13,814,127,42,46,1246,1.10,365.0,46.0,33.0,411.0,81.0,212.0,72.0,01:06,89.0


In [52]:
df_JLFN_fitbit.describe()

,calories,steps,dist,floors,sedant,active_light,active_fair,active_very,active_cals,sleep_start_hr,sleep_minutesAsleep,sleep_minutesAwake,sleep_awakeningsCount,sleep_timeInBed,sleep_REM,sleep_light,sleep_deep,sleep_efficiency
count,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,632.000000,608.000000,608.000000,608.000000,608.000000,146.000000,146.000000,146.000000,632.000000
mean,2785.796053,9996.291667,7.426612,16.007675,896.245614,173.312500,28.087719,34.150219,1141.763158,3.045696,414.856908,45.718750,9.277961,462.055921,69.184932,234.349315,82.253425,90.117089
std,494.504571,4458.903555,3.329586,12.313458,340.523399,110.683546,25.495506,34.596632,719.207967,6.158307,140.408210,23.119734,12.809209,155.389960,23.063631,60.382755,22.045974,3.822391
min,1622.000000,0.000000,0.000000,0.000000,329.000000,0.000000,0.000000,0.000000,0.000000,0.020000,9.000000,0.000000,0.000000,10.000000,18.000000,146.000000,19.000000,78.000000
25%,2550.000000,7983.750000,5.920000,4.000000,652.750000,88.750000,3.000000,1.000000,572.250000,0.630000,359.000000,31.000000,2.000000,404.000000,56.000000,202.000000,68.000000,87.000000
50%,2823.000000,10437.500000,7.700000,17.000000,745.500000,203.500000,25.000000,25.000000,1314.000000,0.950000,386.500000,44.000000,3.000000,430.000000,65.500000,223.000000,83.000000,90.000000
75%,3099.500000,12765.250000,9.535000,23.000000,1249.000000,252.000000,44.000000,57.000000,1662.250000,1.655000,458.000000,57.000000,8.000000,506.250000,81.000000,251.000000,92.000000,93.000000
max,5994.000000,23763.000000,17.860000,111.000000,1440.000000,451.000000,139.000000,213.000000,3058.000000,23.980000,971.000000,166.000000,81.000000,1120.000000,160.000000,591.000000,192.000000,100.000000


In [53]:
df_JLFN_fitbit.isnull().sum() #Note null values in the variables

date                       0
calories                   0
steps                      0
dist                       0
floors                     0
sedant                     0
active_light               0
active_fair                0
active_very                0
active_cals                0
sleep_start_hr           280
sleep_minutesAsleep      304
sleep_minutesAwake       304
sleep_awakeningsCount    304
sleep_timeInBed          304
sleep_REM                766
sleep_light              766
sleep_deep               766
sleep_start              280
sleep_efficiency         280
dtype: int64

In [54]:
df_JLFN_fitbit.dropna().describe()# if I delete null values but reduce the records to 146

,calories,steps,dist,floors,sedant,active_light,active_fair,active_very,active_cals,sleep_start_hr,sleep_minutesAsleep,sleep_minutesAwake,sleep_awakeningsCount,sleep_timeInBed,sleep_REM,sleep_light,sleep_deep,sleep_efficiency
count,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,2882.267123,10774.486301,7.994247,16.280822,693.150685,222.479452,37.157534,42.705479,1466.883562,1.169589,402.116438,66.712329,30.335616,468.883562,69.184932,234.349315,82.253425,85.643836
std,305.669921,3218.327066,2.433214,11.103117,98.953932,62.740101,22.655650,31.926460,404.432024,0.845845,111.373711,20.392344,9.438204,128.588760,23.063631,60.382755,22.045974,2.242965
min,1697.000000,443.000000,0.330000,0.000000,405.000000,26.000000,0.000000,0.000000,82.000000,0.070000,266.000000,39.000000,13.000000,316.000000,18.000000,146.000000,19.000000,78.000000
25%,2696.250000,8719.250000,6.437500,6.000000,631.250000,183.000000,21.000000,16.000000,1207.750000,0.680000,346.250000,55.000000,25.000000,405.250000,56.000000,202.000000,68.000000,84.250000
50%,2838.500000,10681.500000,7.865000,17.500000,706.000000,219.500000,36.000000,39.000000,1427.000000,1.010000,368.500000,62.500000,28.000000,430.500000,65.500000,223.000000,83.000000,86.000000
75%,3097.250000,12304.750000,9.230000,23.750000,759.000000,261.000000,48.750000,64.000000,1734.250000,1.415000,417.500000,72.750000,33.750000,484.500000,81.000000,251.000000,92.000000,87.000000
max,3954.000000,22226.000000,17.860000,75.000000,957.000000,371.000000,116.000000,147.000000,2766.000000,6.000000,954.000000,166.000000,81.000000,1120.000000,160.000000,591.000000,192.000000,92.000000


In [55]:
#Save final df
#df_JLFN_fitbit.to_csv('Datasets/fitbit/JLFN_fitbit_final.csv')